# Introduction

In many real scenarios, the buying and rating behaviors of customers are associated with temporal information.

Time can be viewed from a recency and forecasting perspective, or from a contextual (e.g., seasonal) perspective. From a recency perspective, the basic idea is that recent ratings are more important than older ratings. In the contextual perspective, various periodic aspects, such as season or month, may be used.

When time is viewed as a continuous variable, the recommendations are often created as functions of time. The temporal context can be viewed from a periodic, recency, or modeling point of view.

Time can be treated as a modeling variable by explicitly expressing the predicted ratings as a function of time. The parameters of this function can be learned in a data-driven manner by minimizing the squared error of the predicted ratings with respect to the observed ratings. An example of such a model is time-SVD++, which expresses the predicted ratings as a function of temporally parameterized biases and factor matrices. 

# Temporal Collaborative Filtering

Temporal information can be used in one of two ways in order to improve the effectiveness of prediction:
1. Recency-based models: Some models consider recent ratings more important than older ratings. In these cases, window-based and decay-based models are used for more accurate prediction.
2. Periodic context-based models: In periodic context-based models, the specific property of a period, such as the time at the level of specificity of the hour, day, week, month, or sea- son, is used to perform the recommendation. 
3. Models that explicitly use time as an independent variable: A recent approach, referred to as time-SVD++, uses time as an independent variable within the modeling process. 

## Recency-Based Models

### Decay-Based Methods

In decay-based methods, a time-stamp $t_{uj}$ is associated with each observed rating of user $u$ and item $j$ in the $m \times n$ ratings matrix $R$. It is assumed that all recommendations should be made at a future time $t_f$ . This future time is also referred to as the target time. Then, the weight $w_{uj}(t_f)$ of the rating $r_{uj}$ at target time $t_f$ is defined with the use of a decay function, that penalizes larger distances between $t_{uj}$ and $t_f$ . A decay function is the exponential function: 
$$ w_{uj}(t_f) = exp[-\lambda (t_f - t_{uj})]$$

The decay-rate $\lambda$ is a user-defined parameter that regulates the importance of time. Larger values of $\lambda$ de-emphasize older ratings to a greater degree.

Then:
$$ \hat{r}_{uj} (t_f) = \mu_u + \dfrac{\sum_{v \in P_u(j)} w_{vj} (t_f) .Sim(u, v).(r_{vj} - \mu_v)}{\sum_{v \in P_u(j)} w_{vj} (t_f) .|Sim(u, v)|} $$

Here, $P_u(j)$ represents the $k$ closest users to user $u$ that have specified ratings for item $j$. The optimal value of λ can be learned using cross-validation methods,

#### Example in MovieLens

In [1]:
import numpy as np
import tensorflow as tf
import sklearn
import csv
import pandas as pd
from datetime import datetime

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os

dir_path = os.path.abspath(os.path.join('', os.pardir))

#### Get data

In [3]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(os.path.join(dir_path, 'data/ml-100k/u.data'), names=names, sep='\t')

In [4]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]

In [5]:
nan = np.nan

time_matrix = np.zeros((n_users, n_items)) * nan
ratings_matrix = np.zeros((n_users, n_items)) * nan

for line in df.itertuples():
    ratings_matrix[line[1]-1, line[2]-1] = line[3]
    time_matrix[line[1]-1, line[2]-1] = line[4]

#### Get test data
We need to get the test data with the newest time.

In [6]:
flatten_time_matrix = np.reshape(time_matrix, -1)

In [7]:
sorted_time_indices = np.argsort(flatten_time_matrix) # decreasing

In [8]:
flatten_test_indices = sorted_time_indices[:5000]
flatten_train_indices = sorted_time_indices[5000:]

In [9]:
def get_indices(flatten_indices):
    X = []
    Y = []
    
    for indices in flatten_indices:
        x = indices // n_items
        y = indices % n_items
        
        X.append(x)
        Y.append(y)
        
    return [tuple(X), tuple(Y)]

In [10]:
test_indices = get_indices(flatten_test_indices)
train_indices = get_indices(flatten_train_indices)

In [11]:
def get_ratings_matrix(original_matrix, indices):
    shape = np.shape(original_matrix)
    matrix = np.ones(shape) * nan
    
    for i in range(len(indices[0])):
        x = indices[0][i]
        y = indices[1][i]
        
        matrix[x][y] = original_matrix[x][y]
    
    return matrix

In [12]:
train_ratings_matrix = get_ratings_matrix(ratings_matrix, train_indices)
test_ratings_matrix = get_ratings_matrix(ratings_matrix, test_indices)

train_time_matrix = get_ratings_matrix(time_matrix, train_indices)
test_time_matrix = get_ratings_matrix(time_matrix, test_indices)

In [14]:
# indices for vector
def specified_rating_indices(u):
    return np.where(np.isfinite(u))

In [15]:
# mean rating for each user i using his specified rating
def mean(u):
    # may use specified_rating_indices but use more time
    specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]
    if np.shape(specified_ratings)[0] == 0: return nan
    m = sum(specified_ratings)/np.shape(specified_ratings)[0]
    return m

In [16]:
def all_user_mean_ratings(ratings_matrix):
    return np.array([mean(ratings_matrix[u, :]) for u in range(ratings_matrix.shape[0])])

In [17]:
def get_mean_centered_ratings_matrix(ratings_matrix):
    users_mean_rating = all_user_mean_ratings(ratings_matrix)
    mean_centered_ratings_matrix = ratings_matrix - np.reshape(users_mean_rating, [-1, 1])
    return mean_centered_ratings_matrix

In [18]:
mean_centered_ratings_matrix = get_mean_centered_ratings_matrix(train_ratings_matrix)

In [19]:
def pearson(u, v):
    mean_u = mean(u)
    mean_v = mean(v)
    
    specified_rating_indices_u = set(specified_rating_indices(u)[0])
    specified_rating_indices_v = set(specified_rating_indices(v)[0])
    
    mutually_specified_ratings_indices = specified_rating_indices_u.intersection(specified_rating_indices_v)
    mutually_specified_ratings_indices = list(mutually_specified_ratings_indices)
    
    u_mutually = u[mutually_specified_ratings_indices]
    v_mutually = v[mutually_specified_ratings_indices]
      
    centralized_mutually_u = u_mutually - mean_u
    centralized_mutually_v = v_mutually - mean_v
#     print(np.sqrt(np.sum(np.square(centralized_mutually_u))))

    result = np.sum(np.multiply(centralized_mutually_u, centralized_mutually_v)) 
    result = result / (np.sqrt(np.sum(np.square(centralized_mutually_u))) * np.sqrt(np.sum(np.square(centralized_mutually_v))))
    
    return result

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from surprise import similarities

In [21]:
def mean_centered(u):
    return u - mean(u)

In [22]:
def get_user_similarity_value_for(u_index, ratings_matrix, func):
    user_ratings = ratings_matrix[u_index, :]
    similarity_value = np.array([func(ratings_matrix[i, :], user_ratings) for i in range(ratings_matrix.shape[0])])
    return similarity_value

In [23]:
from tqdm import tqdm
def get_user_similarity_matrix(ratings_matrix, func):
    similarity_matrix = []
    for u_index in tqdm(range(ratings_matrix.shape[0])):
        similarity_value = get_user_similarity_value_for(u_index, ratings_matrix, func)
        similarity_matrix.append(similarity_value)
    return np.array(similarity_matrix)
    

In [24]:
user_similarity_matrix = get_user_similarity_matrix(train_ratings_matrix, pearson)

100%|██████████| 943/943 [01:41<00:00,  9.26it/s]


In [25]:
users_mean_rating = all_user_mean_ratings(train_ratings_matrix)

In [26]:
def diff_month(d1, d2):
    d1 = datetime.fromtimestamp(d1)
    d2 = datetime.fromtimestamp(d2)
    return (d1.year - d2.year) * 12 + d1.month - d2.month

In [27]:
# get weight by diff of months

def weight_for_rating(lambda_param, rating_time, current_time):
    months = diff_month(rating_time, current_time)
    return np.exp(-lambda_param*months)

In [74]:
def predict(u_index, i_index, k):
    
    similarity_value = user_similarity_matrix[u_index]
    sorted_users_similar = np.argsort(similarity_value)
    sorted_users_similar = np.flip(sorted_users_similar, axis=0)
        
    # only for this item
    users_rated_item = specified_rating_indices(ratings_matrix[:, i_index])[0]

    set_2 = frozenset(users_rated_item)
    ranked_similar_user_rated_item = [u for u in sorted_users_similar if u in set_2] 
    
    if k < len(ranked_similar_user_rated_item):
        top_k_similar_user = ranked_similar_user_rated_item[0:k]   
    else:
        top_k_similar_user = np.array(ranked_similar_user_rated_item)
            
    # replace with mean_centered for user
    
    ratings_in_item = mean_centered_ratings_matrix[:, i_index]
    top_k_ratings = ratings_in_item[top_k_similar_user]
    
    top_k_similarity_value = similarity_value[top_k_similar_user]

    r_hat = users_mean_rating[u_index] + np.sum(top_k_ratings * top_k_similarity_value)/np.sum(np.abs(top_k_similarity_value))
    return r_hat

In [75]:
def get_predicted_ratings_matrix():
    predicted_ratings = []
    for u_index in tqdm(range(n_users)):
        user_ratings = []
        for i_index in range(n_items):
#             rating = ratings_matrix[u_index][i_index]
#             if np.isnan(rating):
            rating = predict(u_index, i_index, 100)
            user_ratings.append(rating)
        predicted_ratings.append(user_ratings)
    return predicted_ratings            

In [57]:
predicted_ratings = get_predicted_ratings_matrix()
predicted_ratings = np.array(predicted_ratings)

100%|██████████| 943/943 [05:18<00:00,  2.96it/s]
